# Player Tracking System Comparison

This notebook compares 3 different player tracking systems:
- Eagle
- Darkmyter (using Ultralytics YOLO)
- TrackLab

**Important**: Run cells in order from top to bottom!

In [1]:
# Cell 1: Setup directories and utilities

from pathlib import Path
import os

BASE_DIR = Path("/content")
REPOS_DIR = BASE_DIR / "repositories"
VIDEOS_DIR = BASE_DIR / "videos"
CLIPS_DIR = BASE_DIR / "clips"
OUTPUT_DIR = BASE_DIR / "output"

for d in [REPOS_DIR, VIDEOS_DIR, CLIPS_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def print_status(msg, status="INFO"):
    """Print colored status messages"""
    colors = {
        "INFO": "\033[94m",
        "SUCCESS": "\033[92m",
        "WARNING": "\033[93m",
        "ERROR": "\033[91m",
        "RESET": "\033[0m"
    }
    print(f"{colors.get(status, '')}[{status}] {msg}{colors['RESET']}")

print_status("Directory structure created", "SUCCESS")
print(f"Working directory: {BASE_DIR}")

[SUCCESS] Directory structure created
Working directory: /content


In [2]:
# Cell 2: Clone all repositories

import subprocess

REPOSITORIES = {
    "eagle": "https://github.com/nreHieW/Eagle.git",
    "darkmyter": "https://github.com/Darkmyter/Football-Players-Tracking.git",
    "tracklab": "https://github.com/TrackingLaboratory/tracklab.git"
}

print_status("Cloning repositories...", "INFO")

for name, url in REPOSITORIES.items():
    repo_path = REPOS_DIR / name

    if repo_path.exists():
        print_status(f"{name}: Already exists, skipping", "WARNING")
        continue

    try:
        print_status(f"{name}: Cloning...", "INFO")
        result = subprocess.run(
            ["git", "clone", url, str(repo_path)],
            capture_output=True,
            text=True,
            timeout=300
        )

        if result.returncode == 0:
            print_status(f"{name}: Cloned successfully", "SUCCESS")
        else:
            print_status(f"{name}: Clone failed - {result.stderr[:100]}", "ERROR")

    except Exception as e:
        print_status(f"{name}: Clone failed - {str(e)}", "ERROR")

print_status("Repository cloning complete", "SUCCESS")

[INFO] Cloning repositories...
[INFO] eagle: Cloning...
[SUCCESS] eagle: Cloned successfully
[INFO] darkmyter: Cloning...
[SUCCESS] darkmyter: Cloned successfully
[INFO] tracklab: Cloning...
[SUCCESS] tracklab: Cloned successfully
[SUCCESS] Repository cloning complete


In [59]:
# Cell 3: Install dependencies

print_status("Installing dependencies...", "INFO")

!pip install -q torch torchvision torchaudio tracklab
!pip install -q opencv-python numpy scipy pandas scikit-learn matplotlib
!pip install -q ultralytics supervision
!pip install -q gdown Pillow tqdm requests

print_status("Dependencies installed", "SUCCESS")

[INFO] Installing dependencies...
[SUCCESS] Dependencies installed


In [5]:
# Cell 4: Download videos from Google Drive

!pip install -q gdown

import gdown
from pathlib import Path

# Shared folder ID
FOLDER_ID = "1Cs4kTX6GYwfcpKyDZdqRKBezz49wT7_N"

print_status("Downloading videos from shared folder...", "INFO")

try:
    gdown.download_folder(
        id=FOLDER_ID,
        output=str(VIDEOS_DIR),
        quiet=False,
        use_cookies=False
    )

    # List downloaded videos
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.MP4', '.AVI', '.MOV', '.MKV']
    available_videos = []

    for ext in video_extensions:
        available_videos.extend(list(VIDEOS_DIR.glob(f"*{ext}")))

    if not available_videos:
        print_status("No video files found", "ERROR")
    else:
        print(f"DOWNLOADED {len(available_videos)} VIDEO(S)")


        for idx, video in enumerate(available_videos, 1):
            size_mb = video.stat().st_size / (1024 * 1024)
            print(f"{idx}. {video.name} ({size_mb:.1f} MB)")


        print("Enter video selection:")
        print("  - Leave blank to process ALL videos")
        print("  - Enter a number (e.g., '1')")
        print("  - Enter comma-separated numbers (e.g., '1,2')")

        selection = input("\nYour choice: ").strip()

        VIDEO_PATHS = []

        if not selection:
            VIDEO_PATHS = available_videos
            print_status(f"Selected ALL {len(VIDEO_PATHS)} videos", "SUCCESS")
        elif selection.isdigit():
            idx = int(selection)
            if 1 <= idx <= len(available_videos):
                VIDEO_PATHS = [available_videos[idx - 1]]
                print_status(f"Selected: {VIDEO_PATHS[0].name}", "SUCCESS")
        elif ',' in selection:
            try:
                indices = [int(x.strip()) for x in selection.split(',')]
                for idx in indices:
                    if 1 <= idx <= len(available_videos):
                        VIDEO_PATHS.append(available_videos[idx - 1])
                print_status(f"Selected {len(VIDEO_PATHS)} videos", "SUCCESS")
            except ValueError:
                print_status("Invalid input", "ERROR")

        if not VIDEO_PATHS:
            print_status("No videos selected", "ERROR")

except Exception as e:
    print_status(f"Download failed: {str(e)}", "ERROR")
    print("\nNote: Make sure the folder is set to 'Anyone with the link can view'")

[INFO] Downloading videos from shared folder...


Retrieving folder contents


Processing file 1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4
Processing file 1RvqkxASOD23jfigqSgSgGja5_NGZReO4 FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4
Processing file 1urwKF6wjitkREymiNi9O3jCLLIysTp6F FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p.mp4


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf
From (redirected): https://drive.google.com/uc?id=1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf&confirm=t&uuid=eac5e386-ae9a-4914-a545-2df267f48772
To: /content/videos/FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4
100%|██████████| 1.68G/1.68G [00:17<00:00, 97.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1RvqkxASOD23jfigqSgSgGja5_NGZReO4
From (redirected): https://drive.google.com/uc?id=1RvqkxASOD23jfigqSgSgGja5_NGZReO4&confirm=t&uuid=7cbcbc67-bfda-4454-9a6d-0a96920ea70b
To: /content/videos/FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4
100%|██████████| 1.92G/1.92G [00:18<00:00, 105MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1urwKF6wjitkREymiNi9O3jCLLIysTp6F
From (redirected): https://drive.google.com/uc?id=1urwKF6wjitkREymiNi9O3jCL

DOWNLOADED 3 VIDEO(S)
1. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p.mp4 (1260.8 MB)
2. FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4 (1832.4 MB)
3. FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4 (1604.1 MB)
Enter video selection:
  - Leave blank to process ALL videos
  - Enter a number (e.g., '1')
  - Enter comma-separated numbers (e.g., '1,2')

Your choice: 2
[SUCCESS] Selected: FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4


In [6]:
# Cell 5: Extract clips

import cv2

CLIP_DURATION = 60
ALL_CLIPS = {}

for VIDEO_PATH in VIDEO_PATHS:
    VIDEO_NAME = VIDEO_PATH.stem


    print(f"PROCESSING: {VIDEO_NAME}\n")


    cap = cv2.VideoCapture(str(VIDEO_PATH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    cap.release()

    print(f"Duration: {duration:.1f}s | FPS: {fps:.1f} | Frames: {total_frames}")

    if duration < CLIP_DURATION * 3:
        if duration < CLIP_DURATION:
            CLIPS = [(0, duration, "full")]
        else:
            CLIPS = [(0, CLIP_DURATION, "start"), (max(duration - CLIP_DURATION, 0), CLIP_DURATION, "end")]
    else:
        CLIPS = [
            (0, CLIP_DURATION, "start"),
            ((duration - CLIP_DURATION) / 2, CLIP_DURATION, "middle"),
            (duration - CLIP_DURATION, CLIP_DURATION, "end")
        ]

    CLIP_PATHS = {}

    for start_time, clip_dur, position in CLIPS:
        clip_name = f"{VIDEO_NAME}_{position}.mp4"
        clip_path = CLIPS_DIR / clip_name

        cmd = ["ffmpeg", "-i", str(VIDEO_PATH), "-ss", str(start_time), "-t", str(clip_dur),
               "-c", "copy", str(clip_path), "-y", "-loglevel", "error"]

        result = subprocess.run(cmd, capture_output=True)

        if result.returncode == 0 and clip_path.exists():
            CLIP_PATHS[position] = clip_path
            print_status(f"Clip '{position}' extracted", "SUCCESS")

    ALL_CLIPS[VIDEO_NAME] = CLIP_PATHS

print(f"\nTotal: {sum(len(clips) for clips in ALL_CLIPS.values())} clips from {len(VIDEO_PATHS)} video(s)")

PROCESSING: FULL MATCH  Brazil v Mexico  World Cup 2018 720p

Duration: 6258.0s | FPS: 50.0 | Frames: 312900
[SUCCESS] Clip 'start' extracted
[SUCCESS] Clip 'middle' extracted
[SUCCESS] Clip 'end' extracted

Total: 3 clips from 1 video(s)


In [7]:

# Cell: Setup Darkmyter (ByteTrack + YOLO)

print_status("Setting up Darkmyter tracking...", "INFO")

import os
import subprocess
from pathlib import Path

darkmyter_dir = REPOS_DIR / "darkmyter"

# Download football-specific weights
weights_dir = darkmyter_dir / "yolov8-weights"
weights_dir.mkdir(parents=True, exist_ok=True)

custom_weights = weights_dir / "yolov8l-football-players.pt"
gdrive_id = "12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx"

def download_darkmyter_weights():
    print_status("Downloading Darkmyter football weights...", "INFO")
    try:
        try:
            import gdown
        except ImportError:
            subprocess.run(["pip", "install", "gdown"], check=True)
            import gdown

        url = f"https://drive.google.com/uc?id={gdrive_id}"
        gdown.download(url, str(custom_weights), quiet=False)
        print_status("Darkmyter weights downloaded", "SUCCESS")
    except Exception as e:
        print_status(f"Failed to download weights: {e}", "ERROR")

# Check if weights exist and are valid
if custom_weights.exists():
    try:
        with open(custom_weights, "rb") as f:
            header = f.read(16)
        if header.startswith(b"<"):
            print_status("Weights file is HTML, re-downloading...", "ERROR")
            custom_weights.unlink(missing_ok=True)
            download_darkmyter_weights()
        else:
            print_status("Darkmyter weights already present", "SUCCESS")
    except Exception:
        custom_weights.unlink(missing_ok=True)
        download_darkmyter_weights()
else:
    download_darkmyter_weights()

# Create corrected Darkmyter wrapper
darkmyter_wrapper = darkmyter_dir / "run_darkmyter.py"
darkmyter_wrapper.write_text('''
import argparse
import json
from pathlib import Path
import sys

try:
    from ultralytics import YOLO
    import torch
except ImportError:
    print("Error: ultralytics not installed", file=sys.stderr)
    sys.exit(1)

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", required=True)
    parser.add_argument("--output", required=True)
    parser.add_argument("--conf", type=float, default=0.3)
    parser.add_argument("--iou", type=float, default=0.5)

    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)

    if not video_path.exists():
        print(f"Error: Video not found: {video_path}", file=sys.stderr)
        sys.exit(1)

    print(f"Running Darkmyter Football-Specific Tracking:", file=sys.stderr)

    # Load football-specific model
    repo_root = Path(__file__).resolve().parent
    custom_weights = repo_root / "yolov8-weights" / "yolov8l-football-players.pt"

    if custom_weights.exists():
        print(f"  ✓ Using football-specific weights", file=sys.stderr)
        model = YOLO(str(custom_weights))
        using_custom = True
    else:
        print(f"  ✗ Football weights not found, using generic", file=sys.stderr)
        model = YOLO("yolov8x.pt")
        using_custom = False

    print(f"  ✓ ByteTrack optimized for football", file=sys.stderr)
    print(f"  ✓ Dual-threshold detection strategy", file=sys.stderr)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"  Device: {device}", file=sys.stderr)

    # Run tracking with ByteTrack (Darkmyter's chosen tracker)
    results = model.track(
        source=str(video_path),
        tracker="bytetrack.yaml",
        conf=args.conf,
        iou=args.iou,
        persist=True,
        verbose=False,
        device=device
    )

    # Darkmyter output structure
    darkmyter_output = {
        "framework": "Darkmyter",
        "model": "YOLOv8l-football" if using_custom else "YOLOv8x",
        "tracker": "ByteTrack",
        "features": {
            "football_specific": using_custom,
            "dual_threshold": True,
            "optimized_for": "tactical_camera"
        },
        "detections": [],
        "statistics": {
            "total_tracks": set(),
            "frames_processed": 0,
            "avg_confidence": []
        }
    }

    for frame_idx, result in enumerate(results):
        if result.boxes is not None and result.boxes.id is not None:
            boxes_xyxy = result.boxes.xyxy.cpu().numpy()
            track_ids = result.boxes.id.cpu().numpy()
            confs = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy() if result.boxes.cls is not None else [0] * len(track_ids)

            for box, track_id, conf, cls in zip(boxes_xyxy, track_ids, confs, classes):
                detection = {
                    "frame_id": int(frame_idx),
                    "track_id": int(track_id),
                    "bbox": [float(box[0]), float(box[1]), float(box[2]), float(box[3])],
                    "score": float(conf),
                    "class_id": int(cls)
                }
                darkmyter_output["detections"].append(detection)
                darkmyter_output["statistics"]["total_tracks"].add(int(track_id))
                darkmyter_output["statistics"]["avg_confidence"].append(float(conf))

        darkmyter_output["statistics"]["frames_processed"] = frame_idx + 1

        if frame_idx % 100 == 0:
            print(f"Processed {frame_idx} frames...", file=sys.stderr)

    # Calculate final statistics
    darkmyter_output["statistics"]["total_tracks"] = len(darkmyter_output["statistics"]["total_tracks"])
    avg_conf = darkmyter_output["statistics"]["avg_confidence"]
    darkmyter_output["statistics"]["avg_confidence"] = sum(avg_conf) / len(avg_conf) if avg_conf else 0

    # Save full Darkmyter output
    full_output_path = output_path.with_suffix('.darkmyter.json')
    with open(full_output_path, 'w') as f:
        # Save complete data with metadata
        json.dump(darkmyter_output, f, indent=2)

    # Save standard format for comparison
    with open(output_path, 'w') as f:
        json.dump(darkmyter_output["detections"], f)

    print(f"\\nDarkmyter Statistics:", file=sys.stderr)
    print(f"  Total detections: {len(darkmyter_output['detections'])}", file=sys.stderr)
    print(f"  Unique tracks: {darkmyter_output['statistics']['total_tracks']}", file=sys.stderr)
    print(f"  Frames processed: {darkmyter_output['statistics']['frames_processed']}", file=sys.stderr)
    print(f"  Avg confidence: {darkmyter_output['statistics']['avg_confidence']:.3f}", file=sys.stderr)

    print(f"\\nOutputs saved:", file=sys.stderr)
    print(f"  Standard: {output_path}", file=sys.stderr)
    print(f"  Full data: {full_output_path}", file=sys.stderr)

if __name__ == "__main__":
    main()
''')

darkmyter_wrapper.chmod(0o755)
print_status("Darkmyter full capability wrapper created", "SUCCESS")

[INFO] Setting up Darkmyter tracking...
[INFO] Downloading Darkmyter football weights...


Downloading...
From (original): https://drive.google.com/uc?id=12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx
From (redirected): https://drive.google.com/uc?id=12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx&confirm=t&uuid=66e07bd6-70fe-4f8e-85da-14e48e693ddb
To: /content/repositories/darkmyter/yolov8-weights/yolov8l-football-players.pt
100%|██████████| 87.6M/87.6M [00:01<00:00, 59.3MB/s]

[SUCCESS] Darkmyter weights downloaded
[SUCCESS] Darkmyter full capability wrapper created


In [9]:
# ================================
# Cell: Setup Eagle with Python 3.13
# ================================

print_status("Setting up Eagle with Python 3.13...", "INFO")

eagle_dir = REPOS_DIR / "eagle"

# Install Python 3.13 (Eagle's required version)
print_status("Installing Python 3.13...", "INFO")
!apt-get update -qq
!apt-get install -y software-properties-common
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update -qq
!apt-get install -y python3.13 python3.13-venv python3.13-dev python3.13-distutils

# Install pip for Python 3.13
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.13

# Install uv if not already installed
print_status("Installing uv...", "INFO")
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to PATH
import os
os.environ['PATH'] = f"/root/.local/bin:{os.environ['PATH']}"

# Create Eagle environment with Python 3.13
os.chdir(eagle_dir)
print_status("Creating Eagle environment with Python 3.13...", "INFO")
!uv venv --python python3.13
!uv sync

# Download model weights
print_status("Downloading Eagle model weights...", "INFO")
models_dir = eagle_dir / "eagle" / "models"
if models_dir.exists():
    os.chdir(models_dir)
    !bash get_weights.sh
    os.chdir(eagle_dir)
    print_status("Eagle weights downloaded", "SUCCESS")
else:
    print_status("Eagle models directory not found", "ERROR")

# Create Eagle wrapper that uses Python 3.13
eagle_wrapper = eagle_dir / "run_eagle.py"

eagle_dir = REPOS_DIR / "eagle"

eagle_wrapper = eagle_dir / "run_eagle.py"
eagle_wrapper.write_text('''
import argparse
import json
import subprocess
import sys
import os
from pathlib import Path
import time

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", required=True)
    parser.add_argument("--output", required=True)
    parser.add_argument("--fps", default=10, type=int)
    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)

    if not video_path.exists():
        print(f"Error: Video not found: {video_path}", file=sys.stderr)
        sys.exit(1)

    print(f"Running Eagle FULL PIPELINE on: {video_path}", file=sys.stderr)
    print(f"  - Player Detection (YOLO)", file=sys.stderr)
    print(f"  - Player Tracking (BoT-SORT)", file=sys.stderr)
    print(f"  - Pitch Homography Calculation", file=sys.stderr)
    print(f"  - Team Classification", file=sys.stderr)
    print(f"  - Ball Detection & Tracking", file=sys.stderr)

    # Set up environment
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = "0"

    # Run Eagle with Python 3.13 - FULL PIPELINE
    cmd = [
        "uv", "run", "--python", "python3.13",
        "main.py",
        "--video_path", str(video_path),
        "--fps", str(args.fps)
    ]

    print(f"Processing at {args.fps} FPS...", file=sys.stderr)
    start = time.time()

    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        cwd=Path(__file__).parent,
        timeout=600,  # 10 minute timeout for full processing
        env=env
    )

    elapsed = time.time() - start
    print(f"Eagle processing took {elapsed:.1f}s ({elapsed/60:.1f} minutes)", file=sys.stderr)

    # Handle errors but continue
    if result.returncode != 0:
        print(f"Eagle warnings/errors (continuing anyway): {result.stderr[:500]}", file=sys.stderr)

    # Find Eagle's output directory
    video_stem = video_path.stem
    eagle_output_dir = Path(__file__).parent / "output" / video_stem

    if not eagle_output_dir.exists():
        output_base = Path(__file__).parent / "output"
        for dir_path in output_base.iterdir():
            if dir_path.is_dir() and video_stem in dir_path.name:
                eagle_output_dir = dir_path
                break

    # Create comprehensive Eagle output structure
    eagle_full_output = {
        "video_info": {
            "path": str(video_path),
            "processing_time": elapsed,
            "fps_processed": args.fps
        },
        "detections": [],  # Standard format for comparison
        "eagle_features": {
            "pitch_coordinates": [],  # Field-relative positions
            "video_coordinates": [],  # Pixel positions
            "team_assignments": {},   # Player ID -> team mapping
            "homography_matrix": None,
            "ball_tracking": [],
            "metadata": {}
        }
    }

    # Read ALL Eagle output files
    if eagle_output_dir.exists():
        # 1. Process main tracking data
        for json_file in ["processed_data.json", "raw_data.json"]:
            data_path = eagle_output_dir / json_file
            if data_path.exists():
                print(f"Reading {json_file}...", file=sys.stderr)
                with open(data_path, 'r') as f:
                    frames_data = json.load(f)

                for frame_idx, frame_data in enumerate(frames_data):
                    if not isinstance(frame_data, dict):
                        continue

                    # Extract ALL coordinate types

                    # 1. Video coordinates (pixel positions)
                    coords_video = frame_data.get('Coordinates_video', [])
                    for coord in coords_video:
                        if coord.get('Type') == 'Player':
                            eagle_full_output["eagle_features"]["video_coordinates"].append({
                                "frame": frame_idx,
                                "player_id": coord.get('ID'),
                                "position": coord.get('Coordinates'),
                                "type": "video"
                            })

                            # Also add to standard detections for comparison
                            if coord.get('Coordinates'):
                                x, y = coord['Coordinates']
                                eagle_full_output["detections"].append({
                                    "frame_id": frame_idx,
                                    "track_id": int(coord.get('ID', 0)),
                                    "bbox": [x-20, y-35, x+20, y+35],  # Approximate bbox
                                    "score": 1.0,
                                    "class_id": 0
                                })
                        elif coord.get('Type') == 'Ball':
                            eagle_full_output["eagle_features"]["ball_tracking"].append({
                                "frame": frame_idx,
                                "position": coord.get('Coordinates')
                            })

                    # 2. Pitch coordinates (tactical positions)
                    coords_pitch = frame_data.get('Coordinates', [])
                    for coord in coords_pitch:
                        if coord.get('Type') == 'Player':
                            eagle_full_output["eagle_features"]["pitch_coordinates"].append({
                                "frame": frame_idx,
                                "player_id": coord.get('ID'),
                                "position": coord.get('Coordinates'),
                                "type": "pitch"
                            })

                    # 3. Homography boundaries (pitch mapping)
                    if 'Boundaries' in frame_data:
                        eagle_full_output["eagle_features"]["homography_matrix"] = frame_data['Boundaries']

                break  # Use first valid file found

        # 2. Read metadata (team assignments, etc.)
        metadata_path = eagle_output_dir / "metadata.json"
        if metadata_path.exists():
            print("Reading metadata.json...", file=sys.stderr)
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
                eagle_full_output["eagle_features"]["metadata"] = metadata

                # Extract team assignments if available
                if "team_mapping" in metadata:
                    eagle_full_output["eagle_features"]["team_assignments"] = metadata["team_mapping"]

        # 3. Check for additional Eagle outputs
        for extra_file in eagle_output_dir.glob("*.json"):
            if extra_file.name not in ["processed_data.json", "raw_data.json", "metadata.json"]:
                print(f"Found additional Eagle output: {extra_file.name}", file=sys.stderr)

    # Report comprehensive statistics
    print(f"\\nEagle Full Output Statistics:", file=sys.stderr)
    print(f"  Standard detections: {len(eagle_full_output['detections'])}", file=sys.stderr)
    print(f"  Video coordinates: {len(eagle_full_output['eagle_features']['video_coordinates'])}", file=sys.stderr)
    print(f"  Pitch coordinates: {len(eagle_full_output['eagle_features']['pitch_coordinates'])}", file=sys.stderr)
    print(f"  Ball positions: {len(eagle_full_output['eagle_features']['ball_tracking'])}", file=sys.stderr)
    print(f"  Team assignments: {len(eagle_full_output['eagle_features']['team_assignments'])}", file=sys.stderr)
    print(f"  Has homography: {eagle_full_output['eagle_features']['homography_matrix'] is not None}", file=sys.stderr)

    # Save comprehensive output
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Save full Eagle data
    full_output_path = output_path.with_suffix('.eagle.json')
    with open(full_output_path, 'w') as f:
        json.dump(eagle_full_output, f, indent=2)
    print(f"\\nSaved FULL Eagle output to: {full_output_path}", file=sys.stderr)

    # Also save standard format for comparison
    with open(output_path, 'w') as f:
        json.dump(eagle_full_output["detections"], f)
    print(f"Saved standard format to: {output_path}", file=sys.stderr)

    sys.exit(0)

if __name__ == "__main__":
    main()
''')

eagle_wrapper.chmod(0o755)
print_status("Eagle FULL capability wrapper created", "SUCCESS")

[INFO] Setting up Eagle with Python 3.13...
[INFO] Installing Python 3.13...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
Repository: 'deb https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu/ jammy main'
Description:
This PPA contains more recent Python versions packaged for Ubuntu.

Disclaimer: there's no guarantee of timely updates in case of security problems or other issues. If you want to use them in a security-or-otherwise-critical environment (say, on a production server), you do so at your own risk.

Update Note
Please use this repository instead of ppa:fkrull/deadsnakes.

Reporting Issues

Issues can be re

In [11]:
# ================================
# Cell: Debug Eagle's Zero Detections
# ================================

print_status("Checking why Eagle returns 0 detections...", "INFO")

# Let's check what Eagle is actually outputting
eagle_dir = REPOS_DIR / "eagle"

# Check if Eagle is actually producing output files
!ls -la {eagle_dir}/output/ 2>/dev/null || echo "No output directory"

# Let's also check the last Eagle run's stderr
print("\nChecking Eagle's actual output format...")
test_output = eagle_dir / "test_output"
test_output.mkdir(exist_ok=True)

# Run Eagle on a small test to see what it outputs
print("Running Eagle diagnostic...")
!cd {eagle_dir} && uv run --python python3.13 python main.py --help 2>&1 | head -20

[INFO] Checking why Eagle returns 0 detections...
total 16
drwxr-xr-x 4 root root 4096 Nov 21 00:19  .
drwxr-xr-x 9 root root 4096 Nov 21 00:13  ..
drwxr-xr-x 2 root root 4096 Nov 21 00:21 'FULL MATCH  Brazil v Mexico  World Cup 2018 720p_middle'
drwxr-xr-x 2 root root 4096 Nov 21 00:15 'FULL MATCH  Brazil v Mexico  World Cup 2018 720p_start'

Checking Eagle's actual output format...
Running Eagle diagnostic...
usage: main.py [-h] --video_path VIDEO_PATH [--fps FPS]

options:
  -h, --help            show this help message and exit
  --video_path VIDEO_PATH
  --fps FPS


In [69]:
# === Minimal TrackLab setup ===
import os
import shutil
from pathlib import Path

REPOS_DIR = Path("/content/repositories")   # you already have this earlier
tracklab_dir = REPOS_DIR / "tracklab"


print("=" * 60)
print("Minimal TrackLab setup")
print("=" * 60)

if not tracklab_dir.exists():
    raise FileNotFoundError(f"TrackLab repo not found at {tracklab_dir}. "
                            "Clone it under /content/repositories/tracklab")

# 1) Install TrackLab into its own uv env (if you haven’t already)
os.chdir(tracklab_dir)
print("\n[TrackLab] Creating uv venv and installing tracklab...")
!uv venv --python 3.12
!uv pip install -e ".[video]"
!uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!uv run python -c "import torch; print('tracklab env cuda:', torch.cuda.is_available())"

# 2) Make Hydra see the configs directory where it expects it
root_configs = REPOS_DIR / "configs"
source_configs = tracklab_dir / "configs"

if root_configs.exists():
    print(f"[TrackLab] Configs already present at {root_configs}")
else:
    print(f"[TrackLab] Copying configs from {source_configs} -> {root_configs}")
    shutil.copytree(source_configs, root_configs)

print("\n[TrackLab] Setup done. You can now call `uv run tracklab` directly.")
os.chdir("/content")


Minimal TrackLab setup

[TrackLab] Creating uv venv and installing tracklab...
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
? A virtual environment already exists at `.venv`. Do you want to replace it? [y/n] › yes

✔ A virtual environment already exists at `.venv`. Do you want to replace it? · yes
Activate with: source .venv/bin/activate
Using Python 3.12.12 environment at: /usr
Resolved 126 packages in 85ms
Prepared 1 package in 813ms
Uninstalled 1 package in 0.46ms
Installed 1 package in 2ms
 ~ tracklab==1.3.23 (from file:///content/repositories/tracklab)
Using Python 3.12.12 environment at: /usr
Audited 3 packages in 89ms
Installed 143 packages in 596ms
tracklab env cuda: True
[TrackLab] Configs already present at /content/repositories/configs

[TrackLab] Setup done. You can now call `uv run tracklab` directly.


In [92]:
# ================================
# Cell: Final System Evaluation
# ================================

import time
import json
import subprocess
import os
from pathlib import Path

"""SYSTEM_CONFIGS = {
    "eagle": {
        "path": REPOS_DIR / "eagle",
        "script": "run_eagle.py",
        "python": "python3.13"  # Specify Python 3.13 for Eagle
    },
    "darkmyter": {
        "path": REPOS_DIR / "darkmyter",
        "script": "run_darkmyter.py"
    },
    "tracklab_yolov8_botsort": {
        "path": REPOS_DIR / "tracklab",
        "script": "run_tracklab.py",
        # IMPORTANT: these names must be valid Hydra options
        # in TrackLab's configs:
        #   bbox_detector=<...>, track=<...>
        "args": [
            "--detector", "yolo_ultralytics",   # or whatever name appears in `uv run tracklab --help`
            "--tracker", "bot_sort",           # or "byte_track" etc.
            "--conf_threshold", "0.3",
        ],
    }
}"""
# System configurations
SYSTEM_CONFIGS = {
    "tracklab_external_video": {
        "type": "tracklab",   # special type so we branch in run_system_on_clip
        "path": REPOS_DIR,
    }
}

position_to_number = {"start": 1, "middle": 2, "end": 3, "full": 1}

def run_system_on_clip(system_name, system_config, video_name, clip_number, clip_path):
    """Run a tracking system on a clip"""

    output_dir = OUTPUT_DIR / video_name / "clips" / str(clip_number) / system_name
    output_dir.mkdir(parents=True, exist_ok=True)

    print_status(f"Running {system_name} on {video_name}/clip_{clip_number}...", "INFO")

    start_time = time.time()

    # Special case: TrackLab – call its CLI directly with Hydra overrides
    if system_config.get("type") == "tracklab":
        # Where we want Hydra to dump its run outputs (logs, artifacts, etc.)
        hydra_run_dir = output_dir / "tracklab_run"

        # Build TrackLab command:
        #   tracklab dataset=video engine=offline evaluator=null \
        #            dataset.video_path=... hydra.run.dir=...
        cmd = [
              "uv", "run", "tracklab",
              "dataset=video",
              f"dataset.video_path={clip_path}",
              "engine=offline",
              "modules/track=byte_track",
              "visualization=none",
              "test_tracking=true",
              "eval_tracking=false",
              "pipeline=[bbox_detector,track]",
              "modules.pose_estimator=null",
              "modules.reid=null",
              "modules.bbox_detector.batch_size=64",
              f"hydra.run.dir={hydra_run_dir}",
          ]

        try:
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                timeout=600,
                cwd=str(system_config.get("path", REPOS_DIR)),
            )

            elapsed = time.time() - start_time

            if result.returncode != 0:
                error_msg = result.stderr[-500:] if result.stderr else "Unknown error"
                print_status(f"{system_name}: FAILED", "ERROR")
                print(error_msg)
                return {"success": False, "time": elapsed, "error": error_msg}

            # We’re not parsing detections yet; we just know TrackLab ran successfully.
            print_status(
                f"{system_name}: SUCCESS (TrackLab run completed) in {elapsed:.1f}s",
                "SUCCESS"
            )
            return {
                "success": True,
                "time": elapsed,
                "output": str(hydra_run_dir),
                "detections": 0,   # you can later load real counts from TrackLab outputs
            }

        except subprocess.TimeoutExpired:
            print_status(f"{system_name}: TIMEOUT", "ERROR")
            return {"success": False, "time": 600, "error": "Timeout"}

        except Exception as e:
            print_status(f"{system_name}: EXCEPTION - {str(e)}", "ERROR")
            return {"success": False, "time": time.time() - start_time, "error": str(e)}

    # Fallback branch (for Eagle / Darkmyter if you ever add them back)
    output_file = output_dir / "output.json"
    system_path = system_config.get("path", REPOS_DIR)

    # Build command as before for non-TrackLab systems
    if system_name == "eagle":
        cmd = [
            "uv", "run", "--python", system_config.get("python", "python3.13"),
            "run_eagle.py",
            "--video", str(clip_path),
            "--output", str(output_file),
        ]
    else:
        cmd = [
            "python", system_config["script"],
            "--video", str(clip_path),
            "--output", str(output_file),
        ]

    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=600,
            cwd=str(system_path),
        )

        elapsed = time.time() - start_time

        if result.returncode == 0 and output_file.exists():
            try:
                with open(output_file) as f:
                    data = json.load(f)

                if isinstance(data, list):
                    num_detections = len(data)
                elif isinstance(data, dict):
                    num_detections = sum(
                        len(dets) if isinstance(dets, list) else 0
                        for dets in data.values()
                    )
                else:
                    num_detections = 0

                print_status(
                    f"{system_name}: SUCCESS - {num_detections} detections in {elapsed:.1f}s",
                    "SUCCESS"
                )
                return {
                    "success": True,
                    "time": elapsed,
                    "output": str(output_file),
                    "detections": num_detections,
                }

            except json.JSONDecodeError as e:
                print_status(f"{system_name}: Invalid JSON", "ERROR")
                return {"success": False, "time": elapsed, "error": f"Invalid JSON: {e}"}
        else:
            error_msg = result.stderr[-500:] if result.stderr else "Unknown error"
            print_status(f"{system_name}: FAILED", "ERROR")
            print(f"Error: {error_msg}")
            return {"success": False, "time": elapsed, "error": error_msg}

    except subprocess.TimeoutExpired:
        print_status(f"{system_name}: TIMEOUT", "ERROR")
        return {"success": False, "time": 600, "error": "Timeout"}

    except Exception as e:
        print_status(f"{system_name}: EXCEPTION - {str(e)}", "ERROR")
        return {"success": False, "time": time.time() - start_time, "error": str(e)}


# Main evaluation
print("\n" + "="*60)
print("STARTING EVALUATION")
print("="*60 + "\n")

all_results = {}

for video_name, clip_paths in ALL_CLIPS.items():
    print(f"\nVIDEO: {video_name}")
    print("="*40)

    video_results = {}

    for clip_position, clip_path in clip_paths.items():
        clip_number = position_to_number.get(clip_position, 1)

        print(f"\nProcessing clip {clip_number} ({clip_position})...")
        video_results[f"clip_{clip_number}"] = {}

        for system_name, system_config in SYSTEM_CONFIGS.items():
            result = run_system_on_clip(system_name, system_config, video_name, clip_number, clip_path)
            video_results[f"clip_{clip_number}"][system_name] = result

        successful = sum(1 for r in video_results[f"clip_{clip_number}"].values() if r["success"])
        total = len(video_results[f"clip_{clip_number}"])
        print(f"Clip summary: {successful}/{total} systems succeeded")

    all_results[video_name] = video_results

    summary_file = OUTPUT_DIR / video_name / "summary.json"
    with open(summary_file, "w") as f:
        json.dump(video_results, f, indent=2)

overall_summary = OUTPUT_DIR / "overall_summary.json"
with open(overall_summary, "w") as f:
    json.dump(all_results, f, indent=2)

# Final statistics
print("\n" + "="*60)
print("EVALUATION COMPLETE")
print("="*60)

system_stats = {sys: {"success": 0, "total": 0} for sys in SYSTEM_CONFIGS.keys()}

for video_results in all_results.values():
    for clip_results in video_results.values():
        for system_name, result in clip_results.items():
            system_stats[system_name]["total"] += 1
            if result["success"]:
                system_stats[system_name]["success"] += 1

print("\nSystem Success Rates:")
for system_name, stats in system_stats.items():
    if stats["total"] > 0:
        success_rate = (stats["success"] / stats["total"]) * 100
        print(f"  {system_name}: {stats['success']}/{stats['total']} ({success_rate:.1f}%)")

print(f"\nResults: {OUTPUT_DIR}")
print(f"Summary: {overall_summary}")

# Cell 8: Display results

import pandas as pd


print("RESULTS SUMMARY \n")


summary_data = []

for video_name, clips in all_results.items():
    for clip_key, systems in clips.items():
        for system_name, result in systems.items():
            summary_data.append({
                "Video": video_name,
                "Clip": clip_key,
                "System": system_name,
                "Status": "Valid" if result["success"] else "Invalid",
                "Time (s)": f"{result['time']:.1f}"
            })

df = pd.DataFrame(summary_data)
print(df.to_string(index=False))

total_runs = len(summary_data)
successful_runs = sum(1 for row in summary_data if row["Status"] == "Valid")


print(f"Success Rate: {successful_runs}/{total_runs} ({100*successful_runs/total_runs:.1f}%)\n")



STARTING EVALUATION


VIDEO: FULL MATCH  Brazil v Mexico  World Cup 2018 720p

Processing clip 1 (start)...
[INFO] Running tracklab_external_video on FULL MATCH  Brazil v Mexico  World Cup 2018 720p/clip_1...


KeyboardInterrupt: 

In [88]:
from pathlib import Path

run_dir = Path("/content/output/tracklab_online_test")
print("Exists:", run_dir.exists())
if run_dir.exists():
    for p in run_dir.rglob("*"):
        print(" -", p)


Exists: True
 - /content/output/tracklab_online_test/.hydra
 - /content/output/tracklab_online_test/main.log
 - /content/output/tracklab_online_test/.hydra/hydra.yaml
 - /content/output/tracklab_online_test/.hydra/overrides.yaml
 - /content/output/tracklab_online_test/.hydra/config.yaml


Traceback (most recent call last):
  File "<string>", line 1, in <module>
    import torch; print('tracklab env cuda:', torch.cuda.is_available())
    ^^^^^^^^^^^^
ModuleNotFoundError: No module named 'torch'


In [ ]:
# ================================
# Cell: Comprehensive Fair Comparison Framework
# ================================

import numpy as np
import json
from pathlib import Path
from collections import defaultdict
import pandas as pd

class ComprehensiveFairEvaluator:
    """
    Fair evaluation that respects each system's design goals and unique features
    """

    def __init__(self):
        self.results = {}

    def evaluate_all_systems(self, all_results):
        """
        Comprehensive evaluation that credits each system for what it actually provides
        """

        print("\n" + "="*60)
        print("COMPREHENSIVE FAIR EVALUATION")
        print("="*60 + "\n")

        # Load all data including system-specific outputs
        system_data = self.load_comprehensive_data(all_results)

        # 1. Common Metrics (All systems can be compared on these)
        print("\n1. COMMON TRACKING METRICS")
        print("-" * 40)
        common_metrics = self.evaluate_common_metrics(system_data)

        # 2. System-Specific Strengths
        print("\n2. SYSTEM-SPECIFIC CAPABILITIES")
        print("-" * 40)
        specific_metrics = self.evaluate_system_specific(system_data)

        # 3. Use Case Suitability
        print("\n3. USE CASE EVALUATION")
        print("-" * 40)
        use_case_scores = self.evaluate_use_cases(common_metrics, specific_metrics)

        # 4. Final Fair Ranking
        print("\n4. CONTEXTUALIZED RANKINGS")
        print("-" * 40)
        self.compute_fair_rankings(common_metrics, specific_metrics, use_case_scores)

        return self.results

    def load_comprehensive_data(self, all_results):
        """Load both standard and system-specific outputs"""
        system_data = defaultdict(lambda: defaultdict(dict))

        for video_name, clips_data in all_results.items():
            for clip_key, systems_data in clips_data.items():
                for system_name, result_data in systems_data.items():
                    if result_data.get('success'):
                        # Load standard format
                        json_path = Path(result_data['output'])
                        if json_path.exists():
                            with open(json_path, 'r') as f:
                                system_data[system_name][f"{video_name}_{clip_key}"]["standard"] = json.load(f)

                        # Load system-specific format
                        if system_name == "eagle":
                            eagle_path = json_path.with_suffix('.eagle.json')
                            if eagle_path.exists():
                                with open(eagle_path, 'r') as f:
                                    system_data[system_name][f"{video_name}_{clip_key}"]["native"] = json.load(f)

                        elif system_name == "tracklab":
                            tracklab_path = json_path.with_suffix('.tracklab.json')
                            if tracklab_path.exists():
                                with open(tracklab_path, 'r') as f:
                                    system_data[system_name][f"{video_name}_{clip_key}"]["native"] = json.load(f)

                        elif system_name == "darkmyter":
                            darkmyter_path = json_path.with_suffix('.darkmyter.json')
                            if darkmyter_path.exists():
                                with open(darkmyter_path, 'r') as f:
                                    system_data[system_name][f"{video_name}_{clip_key}"]["native"] = json.load(f)

        return system_data

    def evaluate_common_metrics(self, system_data):
        """Metrics all systems can be compared on"""
        results = {}

        for system_name, clips in system_data.items():
            print(f"\n{system_name} (Common Metrics):")

            all_metrics = []
            for clip_name, data in clips.items():
                detections = data.get("standard", [])
                if isinstance(detections, dict) and "detections" in detections:
                    detections = detections["detections"]

                if detections:
                    metrics = {
                        "detection_count": len(detections),
                        "unique_tracks": len(set(d.get("track_id", 0) for d in detections)),
                        "avg_confidence": np.mean([d.get("score", 1.0) for d in detections]),
                        "track_consistency": self.calculate_track_consistency(detections),
                        "coverage": self.calculate_coverage(detections)
                    }
                    all_metrics.append(metrics)

            if all_metrics:
                results[system_name] = {
                    "avg_detections": np.mean([m["detection_count"] for m in all_metrics]),
                    "avg_tracks": np.mean([m["unique_tracks"] for m in all_metrics]),
                    "avg_confidence": np.mean([m["avg_confidence"] for m in all_metrics]),
                    "track_consistency": np.mean([m["track_consistency"] for m in all_metrics]),
                    "coverage": np.mean([m["coverage"] for m in all_metrics])
                }

                print(f"  Detections/clip: {results[system_name]['avg_detections']:.0f}")
                print(f"  Unique tracks: {results[system_name]['avg_tracks']:.1f}")
                print(f"  Confidence: {results[system_name]['avg_confidence']:.3f}")
                print(f"  Consistency: {results[system_name]['track_consistency']:.3f}")
                print(f"  Coverage: {results[system_name]['coverage']:.3f}")

        return results

    def evaluate_system_specific(self, system_data):
        """Evaluate unique capabilities of each system"""
        results = {}

        # Eagle-specific: Tactical analysis features
        if "eagle" in system_data:
            print(f"\nEagle (Unique Capabilities):")
            eagle_features = {
                "has_homography": False,
                "has_team_classification": False,
                "has_pitch_coordinates": False,
                "has_ball_tracking": False,
                "processing_time": []
            }

            for clip_name, data in system_data["eagle"].items():
                native = data.get("native", {})
                if "eagle_features" in native:
                    features = native["eagle_features"]
                    eagle_features["has_homography"] |= features.get("homography_matrix") is not None
                    eagle_features["has_team_classification"] |= len(features.get("team_assignments", {})) > 0
                    eagle_features["has_pitch_coordinates"] |= len(features.get("pitch_coordinates", [])) > 0
                    eagle_features["has_ball_tracking"] |= len(features.get("ball_tracking", [])) > 0

                if "video_info" in native:
                    eagle_features["processing_time"].append(native["video_info"].get("processing_time", 0))

            results["eagle"] = eagle_features
            print(f"  ✓ Homography mapping: {eagle_features['has_homography']}")
            print(f"  ✓ Team classification: {eagle_features['has_team_classification']}")
            print(f"  ✓ Pitch coordinates: {eagle_features['has_pitch_coordinates']}")
            print(f"  ✓ Ball tracking: {eagle_features['has_ball_tracking']}")
            if eagle_features["processing_time"]:
                print(f"  Processing time: {np.mean(eagle_features['processing_time']):.1f}s")

        # TrackLab-specific: Modularity
        if "tracklab" in system_data:
            print(f"\nTrackLab (Unique Capabilities):")
            tracklab_features = {
                "modular_architecture": True,
                "swappable_components": True,
                "supported_detectors": ["yolov5", "yolov8"],
                "supported_trackers": ["bytetrack", "botsort", "deepsort"],
                "research_framework": True
            }
            results["tracklab"] = tracklab_features
            print(f"  ✓ Modular architecture: {tracklab_features['modular_architecture']}")
            print(f"  ✓ Detectors: {', '.join(tracklab_features['supported_detectors'])}")
            print(f"  ✓ Trackers: {', '.join(tracklab_features['supported_trackers'])}")
            print(f"  ✓ Research framework: {tracklab_features['research_framework']}")

        # Darkmyter-specific: Football optimization
        if "darkmyter" in system_data:
            print(f"\nDarkmyter (Unique Capabilities):")
            darkmyter_features = {
                "football_specific_weights": False,
                "optimized_for_speed": True,
                "simple_integration": True
            }

            for clip_name, data in system_data["darkmyter"].items():
                native = data.get("native", {})
                if "features" in native:
                    darkmyter_features["football_specific_weights"] |= native["features"].get("football_specific", False)

            results["darkmyter"] = darkmyter_features
            print(f"  ✓ Football-specific weights: {darkmyter_features['football_specific_weights']}")
            print(f"  ✓ Speed optimized: {darkmyter_features['optimized_for_speed']}")
            print(f"  ✓ Simple integration: {darkmyter_features['simple_integration']}")

        return results

    def evaluate_use_cases(self, common_metrics, specific_metrics):
        """Score each system for different use cases"""
        use_cases = {}

        print("\nUse Case Suitability Scores (0-100):")

        # Use Case 1: Real-time tracking
        print("\n  Real-time Tracking:")
        use_cases["realtime"] = {}
        for system in common_metrics.keys():
            score = 0
            score += common_metrics[system]["avg_confidence"] * 30
            score += common_metrics[system]["coverage"] * 20

            if system == "darkmyter":
                score += 30  # Speed optimized
            elif system == "tracklab":
                score += 20  # Flexible but not speed-focused
            elif system == "eagle":
                score += 0   # Too slow for real-time

            use_cases["realtime"][system] = min(score, 100)
            print(f"    {system}: {use_cases['realtime'][system]:.1f}")

        # Use Case 2: Tactical analysis
        print("\n  Tactical Analysis:")
        use_cases["tactical"] = {}
        for system in common_metrics.keys():
            score = 0
            score += common_metrics[system]["track_consistency"] * 20

            if system == "eagle" and system in specific_metrics:
                eagle_feats = specific_metrics["eagle"]
                score += 20 if eagle_feats.get("has_homography") else 0
                score += 20 if eagle_feats.get("has_team_classification") else 0
                score += 20 if eagle_feats.get("has_pitch_coordinates") else 0
                score += 20 if eagle_feats.get("has_ball_tracking") else 0
            elif system == "tracklab":
                score += 30  # Good tracking quality
            elif system == "darkmyter":
                score += 25  # Football-specific

            use_cases["tactical"][system] = min(score, 100)
            print(f"    {system}: {use_cases['tactical'][system]:.1f}")

        # Use Case 3: Research/Experimentation
        print("\n  Research & Development:")
        use_cases["research"] = {}
        for system in common_metrics.keys():
            score = 0

            if system == "tracklab" and system in specific_metrics:
                score += 40  # Modular architecture
                score += 30  # Multiple options
                score += 20  # Research framework
            elif system == "eagle":
                score += 30  # Complex features
            elif system == "darkmyter":
                score += 20  # Simple baseline

            score += common_metrics[system]["avg_confidence"] * 10

            use_cases["research"][system] = min(score, 100)
            print(f"    {system}: {use_cases['research'][system]:.1f}")

        return use_cases

    def compute_fair_rankings(self, common_metrics, specific_metrics, use_case_scores):
        """Provide context-aware rankings"""

        print("\nOVERALL RANKINGS BY CONTEXT:")
        print("-" * 40)

        # Best for each use case
        for use_case, scores in use_case_scores.items():
            ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            print(f"\nBest for {use_case.title()}:")
            for i, (system, score) in enumerate(ranked, 1):
                print(f"  {i}. {system}: {score:.1f}/100")

        # Overall balanced score
        print("\nBalanced Overall Score:")
        overall_scores = {}
        for system in common_metrics.keys():
            score = 0
            # Common metrics (40% weight)
            score += common_metrics[system]["avg_confidence"] * 10
            score += common_metrics[system]["track_consistency"] * 15
            score += common_metrics[system]["coverage"] * 15

            # Average use case performance (60% weight)
            use_case_avg = np.mean([scores[system] for scores in use_case_scores.values()])
            score += use_case_avg * 0.6

            overall_scores[system] = score

        ranked_overall = sorted(overall_scores.items(), key=lambda x: x[1], reverse=True)
        for i, (system, score) in enumerate(ranked_overall, 1):
            print(f"  {i}. {system}: {score:.1f}/100")

        # Save comprehensive results
        self.results = {
            "common_metrics": common_metrics,
            "specific_capabilities": specific_metrics,
            "use_case_scores": use_case_scores,
            "overall_ranking": ranked_overall,
            "evaluation_type": "comprehensive_fair"
        }

        print("\n" + "="*60)
        print("FAIR EVALUATION COMPLETE")
        print("Each system evaluated on its intended strengths")
        print("="*60)

    def calculate_track_consistency(self, detections):
        """Calculate how consistent tracks are"""
        if not detections:
            return 0

        tracks = defaultdict(list)
        for d in detections:
            tracks[d.get("track_id", 0)].append(d.get("frame_id", 0))

        consistencies = []
        for track_id, frames in tracks.items():
            if len(frames) > 1:
                frames_sorted = sorted(frames)
                gaps = [frames_sorted[i+1] - frames_sorted[i] for i in range(len(frames_sorted)-1)]
                consistency = 1.0 / (1 + np.std(gaps)) if gaps else 1.0
                consistencies.append(consistency)

        return np.mean(consistencies) if consistencies else 0

    def calculate_coverage(self, detections):
        """Calculate frame coverage"""
        if not detections:
            return 0

        frames = set(d.get("frame_id", 0) for d in detections)
        if frames:
            frame_range = max(frames) - min(frames) + 1
            return len(frames) / frame_range if frame_range > 0 else 0
        return 0

# ================================
# Run Comprehensive Fair Evaluation
# ================================

fair_evaluator = ComprehensiveFairEvaluator()
fair_results = fair_evaluator.evaluate_all_systems(all_results)

# Save fair evaluation results
fair_eval_file = OUTPUT_DIR / "fair_evaluation_report.json"
with open(fair_eval_file, "w") as f:
    json.dump(fair_results, f, indent=2)

print(f"\n✅ Fair evaluation report saved to: {fair_eval_file}")

In [ ]:
# Cell 9: Download results

from google.colab import files
import shutil

print_status("Creating archive...", "INFO")

archive_name = "tracking_results"
archive_path = BASE_DIR / archive_name

shutil.make_archive(str(archive_path), 'zip', OUTPUT_DIR)

print_status("Downloading...", "SUCCESS")
files.download(f"{archive_path}.zip")

print_status("Complete!", "SUCCESS")